# 🖼️ OpenCV Review for Computer Vision

**Mục tiêu:** Ôn tập OpenCV cho image preprocessing và CV tasks

**Nội dung:**
- Image I/O & basic operations
- Color spaces & conversions
- Geometric transformations
- Image filtering
- Edge detection
- Contours & shapes
- Basic object detection

**Level:** Intermediate

---

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

print(f"OpenCV version: {cv2.__version__}")

# Helper function to display images
def show_image(img, title='Image', cmap=None):
    plt.figure(figsize=(8, 6))
    if len(img.shape) == 3:
        # BGR to RGB for matplotlib
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img, cmap=cmap)
    plt.title(title, fontsize=14, fontweight='bold')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

def show_images(images, titles, cmap=None):
    n = len(images)
    fig, axes = plt.subplots(1, n, figsize=(5*n, 5))
    if n == 1:
        axes = [axes]
    for img, title, ax in zip(images, titles, axes):
        if len(img.shape) == 3:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        ax.imshow(img, cmap=cmap)
        ax.set_title(title, fontweight='bold')
        ax.axis('off')
    plt.tight_layout()
    plt.show()

---

## 1. Image I/O & Basic Operations

In [ ]:
# Create synthetic image (since we don't have actual image files)
# In practice: img = cv2.imread('image.jpg')

# Create color image (H, W, 3)
h, w = 300, 400
img = np.zeros((h, w, 3), dtype=np.uint8)

# Draw some shapes
cv2.rectangle(img, (50, 50), (150, 150), (0, 255, 0), -1)  # Green square
cv2.circle(img, (300, 150), 50, (255, 0, 0), -1)  # Blue circle
cv2.line(img, (0, 250), (400, 250), (0, 0, 255), 2)  # Red line
cv2.putText(img, 'OpenCV', (150, 280), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

print(f"Image shape: {img.shape} (H, W, C)")
print(f"Image dtype: {img.dtype}")
print(f"Value range: [{img.min()}, {img.max()}]")

# Basic info
height, width, channels = img.shape
print(f"\nDimensions: {width}x{height}, Channels: {channels}")

show_image(img, 'Original Image')

# Save image
# cv2.imwrite('output.jpg', img)

print("\n💡 OpenCV uses BGR, not RGB!")
print("   When displaying with matplotlib, convert to RGB")

## 2. Color Spaces

In [ ]:
# Convert between color spaces
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

print(f"Gray shape: {gray.shape}")
print(f"HSV shape: {hsv.shape}")

show_images([img, gray], ['Original (BGR)', 'Grayscale'], cmap='gray')

# Split channels
b, g, r = cv2.split(img)
h, s, v = cv2.split(hsv)

print("\nChannel info:")
print(f"  Blue channel: {b.shape}")
print(f"  Hue channel: {h.shape}")

# Merge channels
merged = cv2.merge([b, g, r])
print(f"  Merged: {merged.shape}")

### 💡 Common Color Spaces

- **BGR**: OpenCV default (Blue, Green, Red)
- **RGB**: Standard (Red, Green, Blue)
- **Grayscale**: Single channel intensity
- **HSV**: Hue, Saturation, Value (good for color-based segmentation)
- **LAB**: Lightness, A, B (perceptually uniform)

## 3. Geometric Transformations

In [ ]:
# Resize
resized = cv2.resize(img, (200, 150))  # (width, height)
print(f"Resized: {img.shape} -> {resized.shape}")

# Resize with interpolation
upscaled = cv2.resize(img, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_LINEAR)
downscaled = cv2.resize(img, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)

# Rotation
center = (width // 2, height // 2)
M = cv2.getRotationMatrix2D(center, 45, 1.0)  # 45 degrees, scale=1
rotated = cv2.warpAffine(img, M, (width, height))

# Flip
flipped_h = cv2.flip(img, 1)  # Horizontal
flipped_v = cv2.flip(img, 0)  # Vertical

# Translation
M_translate = np.float32([[1, 0, 50], [0, 1, 30]])  # Shift by (50, 30)
translated = cv2.warpAffine(img, M_translate, (width, height))

show_images([img, rotated, flipped_h], 
            ['Original', 'Rotated 45°', 'Flipped Horizontal'])

### Cropping

In [ ]:
# Crop using array slicing
cropped = img[50:200, 100:300]  # [y1:y2, x1:x2]

print(f"Cropped: {cropped.shape}")
show_image(cropped, 'Cropped Region')

## 4. Image Filtering

In [ ]:
# Add noise to demonstrate filtering
noisy = img.copy()
noise = np.random.randn(*img.shape) * 25
noisy = np.clip(noisy + noise, 0, 255).astype(np.uint8)

# Gaussian blur (remove noise)
blurred = cv2.GaussianBlur(noisy, (5, 5), 0)

# Median blur (good for salt-and-pepper noise)
median = cv2.medianBlur(noisy, 5)

# Bilateral filter (edge-preserving)
bilateral = cv2.bilateralFilter(noisy, 9, 75, 75)

show_images([img, noisy, blurred, median], 
            ['Original', 'Noisy', 'Gaussian Blur', 'Median Filter'])

print("Filter comparison:")
print("  Gaussian: Fast, smooths everything")
print("  Median: Good for salt-and-pepper noise")
print("  Bilateral: Preserves edges (slower)")

### Sharpening

In [ ]:
# Sharpen kernel
kernel_sharpen = np.array([[-1,-1,-1],
                            [-1, 9,-1],
                            [-1,-1,-1]])

sharpened = cv2.filter2D(img, -1, kernel_sharpen)

show_images([img, sharpened], ['Original', 'Sharpened'])

## 5. Edge Detection

In [ ]:
# Canny edge detection
edges_canny = cv2.Canny(gray, 50, 150)  # threshold1=50, threshold2=150

# Sobel edge detection
sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)  # X gradient
sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)  # Y gradient
sobel_combined = np.sqrt(sobelx**2 + sobely**2)
sobel_combined = np.uint8(np.clip(sobel_combined, 0, 255))

# Laplacian
laplacian = cv2.Laplacian(gray, cv2.CV_64F)
laplacian = np.uint8(np.abs(laplacian))

show_images([gray, edges_canny, sobel_combined, laplacian],
            ['Original', 'Canny', 'Sobel', 'Laplacian'],
            cmap='gray')

print("Edge detection methods:")
print("  Canny: Multi-stage, best overall")
print("  Sobel: Gradient-based, directional")
print("  Laplacian: Second derivative, sensitive to noise")

## 6. Thresholding & Segmentation

In [ ]:
# Binary threshold
_, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

# Adaptive threshold (local)
adaptive = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                 cv2.THRESH_BINARY, 11, 2)

# Otsu's threshold (automatic)
_, otsu = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

show_images([gray, binary, adaptive, otsu],
            ['Original', 'Binary', 'Adaptive', 'Otsu'],
            cmap='gray')

print("Thresholding:")
print("  Binary: Fixed threshold")
print("  Adaptive: Local threshold (good for varying illumination)")
print("  Otsu: Automatic threshold selection")

## 7. Contours & Shape Detection

In [ ]:
# Find contours
contours, hierarchy = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

print(f"Found {len(contours)} contours")

# Draw contours
img_contours = img.copy()
cv2.drawContours(img_contours, contours, -1, (0, 255, 0), 2)  # Draw all in green

# Analyze largest contour
if len(contours) > 0:
    largest = max(contours, key=cv2.contourArea)
    
    # Contour properties
    area = cv2.contourArea(largest)
    perimeter = cv2.arcLength(largest, True)
    
    # Bounding box
    x, y, w_box, h_box = cv2.boundingRect(largest)
    cv2.rectangle(img_contours, (x, y), (x+w_box, y+h_box), (255, 0, 0), 2)
    
    # Moments (for centroid)
    M = cv2.moments(largest)
    if M['m00'] != 0:
        cx = int(M['m10'] / M['m00'])
        cy = int(M['m01'] / M['m00'])
        cv2.circle(img_contours, (cx, cy), 5, (0, 0, 255), -1)
    
    print(f"\nLargest contour:")
    print(f"  Area: {area:.0f}")
    print(f"  Perimeter: {perimeter:.1f}")
    print(f"  Centroid: ({cx}, {cy})")

show_images([binary, img_contours], ['Binary', 'Contours'], cmap='gray')

## 8. Morphological Operations

In [ ]:
# Kernel for morphology
kernel = np.ones((5, 5), np.uint8)

# Erosion (shrink white regions)
erosion = cv2.erode(binary, kernel, iterations=1)

# Dilation (expand white regions)
dilation = cv2.dilate(binary, kernel, iterations=1)

# Opening (erosion then dilation, removes noise)
opening = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)

# Closing (dilation then erosion, fills holes)
closing = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)

show_images([binary, erosion, dilation, opening],
            ['Original', 'Erosion', 'Dilation', 'Opening'],
            cmap='gray')

print("Morphological ops:")
print("  Erosion: Shrink objects, remove noise")
print("  Dilation: Expand objects, fill gaps")
print("  Opening: Remove small objects")
print("  Closing: Fill small holes")

## 9. Feature Detection (Bonus)

In [ ]:
# Harris corner detection
gray_float = np.float32(gray)
corners = cv2.cornerHarris(gray_float, 2, 3, 0.04)

# Dilate to mark corners
corners = cv2.dilate(corners, None)

# Mark corners in red
img_corners = img.copy()
img_corners[corners > 0.01 * corners.max()] = [0, 0, 255]

show_image(img_corners, 'Harris Corners')

print("✅ Corner detection useful for:")
print("  - Feature matching")
print("  - Object tracking")
print("  - Image stitching")

---

## 🎯 Key Takeaways

### Essential OpenCV Operations

#### Image I/O
```python
img = cv2.imread('image.jpg')  # Load
cv2.imwrite('output.jpg', img)  # Save
```

#### Color Conversion
```python
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # For matplotlib
```

#### Transformations
```python
resized = cv2.resize(img, (width, height))
rotated = cv2.warpAffine(img, M, (width, height))
cropped = img[y1:y2, x1:x2]
```

#### Filtering
```python
blurred = cv2.GaussianBlur(img, (5, 5), 0)
edges = cv2.Canny(gray, 50, 150)
```

#### Contours
```python
contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(img, contours, -1, (0, 255, 0), 2)
```

### Common Preprocessing for DL

```python
# Typical preprocessing pipeline
img = cv2.imread('image.jpg')

# 1. Resize to model input size
img = cv2.resize(img, (224, 224))

# 2. Convert to RGB (if needed)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# 3. Normalize to [0, 1]
img = img.astype(np.float32) / 255.0

# 4. Add batch dimension
img = np.expand_dims(img, axis=0)
```

### Important Notes

1. **OpenCV uses BGR**, not RGB!
2. **Image shape**: (H, W, C) not (C, H, W)
3. **Dtype matters**: uint8 [0, 255] vs float32 [0, 1]
4. **Coordinate system**: (x, y) = (column, row)
5. **Array indexing**: img[y, x] or img[y1:y2, x1:x2]

### Use Cases in DL

- **Data augmentation**: Resize, rotate, flip
- **Preprocessing**: Normalize, grayscale
- **Feature extraction**: Edges, corners
- **Segmentation**: Thresholding, contours
- **Object detection**: Contours, bounding boxes

---

**Complete!** You now have a comprehensive review of ML/DL libraries.